In [31]:
import requests
from bs4 import BeautifulSoup
import unicodedata
import pandas as pd
import re
import time

In [32]:
data=pd.read_csv("../merged/len3_ndb_blp_DN_firm_val.csv",encoding="utf-8",index_col=0)
data.head()
data["医薬品コード"]=data["医薬品コード"].astype(int).astype(str)
data.loc[data["メーカー名"].isna(),"医薬品名"].value_counts()


C:\Users\hakos\AppData\Local\Temp\ipykernel_17836\2245958985.py:1: DtypeWarning: Columns (118,119,131) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv("../merged/len3_ndb_blp_DN_firm_val.csv",encoding="utf-8",index_col=0)


ジスチグミン臭化物錠　５ｍｇ                     16
Ｄソルビトール                            16
複方ヨード・グリセリン                        15
トルナフタート液　２％                        14
薬用炭                                14
                                   ..
ケトチフェン点鼻液０．０５％「ツルハラ」　６．０４８ｍｇ８ｍＬ     1
クロモグリク酸ナトリウム１９０ｍｇ９．５ｍＬ点鼻液           1
ビホナゾール１％液                           1
カモスタットメシル酸塩１００ｍｇ錠                   1
イトラコナゾール５０ｍｇ錠                       1
Name: 医薬品名, Length: 61, dtype: int64

In [33]:
search_nums=set(data.loc[data["メーカー名"].isna(),"医薬品コード"])
len(search_nums)

62

In [34]:
# test
# ここにあなたのHTMLコードを設定してください
code2firm={}
url="https://www.nichiiko.co.jp/medicine/transitional/measure/"
r=requests.get(url)
soup=BeautifulSoup(r.content,"html.parser")

td_elements = soup.find_all('td', class_="recept-code")
for target_number in search_nums:

    # print(td_elements)

    td_target = None
    for td_element in td_elements:
        if target_number in td_element.text:
            td_target = td_element
            print("文字列が含まれる<td>要素のテキスト:", td_element.text)
    # 対象の<td>要素が見つかった場合
    if td_target:
        # 前の<td>要素を取得
        previous_td = td_target.find_previous_sibling('td')
        previous_td = previous_td.find_previous_sibling('td')
        previous_td = previous_td.find_previous_sibling('td')
        if previous_td:
            code2firm[target_number]=previous_td.text
            print(previous_td.text)


文字列が含まれる<td>要素のテキスト: 620177407611230132
武田テバ薬品
文字列が含まれる<td>要素のテキスト: 620008988622773100
武田テバ薬品
文字列が含まれる<td>要素のテキスト: 621472501622730100
日医工
文字列が含まれる<td>要素のテキスト: 620003996622747800
武田テバファーマ
文字列が含まれる<td>要素のテキスト: 621730103622781600
文字列が含まれる<td>要素のテキスト: 621675401622781600
日医工
文字列が含まれる<td>要素のテキスト: 621373439622781000
武田テバ薬品
文字列が含まれる<td>要素のテキスト: 620882208610461256
武田テバファーマ
文字列が含まれる<td>要素のテキスト: 620006496622782300
武田テバファーマ
文字列が含まれる<td>要素のテキスト: 621705802622781700
前田
文字列が含まれる<td>要素のテキスト: 621407306622773800
日医工岐阜工場
文字列が含まれる<td>要素のテキスト: 621569101622747300
日医工
文字列が含まれる<td>要素のテキスト: 620004237622774100
日医工
文字列が含まれる<td>要素のテキスト: 620564921610461096
日医工
文字列が含まれる<td>要素のテキスト: 620007158622311700
サンノーバ


In [38]:
code2firm
# save as pickle
import pickle
with open("./pickle/code2firm.pkl","wb") as f:
    pickle.dump(code2firm,f)


In [37]:
data.loc[data["メーカー名"].isna(),"メーカー名"]=data.loc[data["メーカー名"].isna(),"医薬品コード"].map(code2firm)
len(set(data.loc[data["メーカー名"].isna(),"医薬品名"]))

47